#### Imports, deck creation and definitions

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata as rd
import random
from collections import OrderedDict
from itertools import combinations
from functools import reduce

In [3]:
def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n-k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

def get_hand_rank(hand):
    """
    Fast hand evaluator courtesy of 
    https://suffe.cool/poker/evaluator.html
    
    Input hand 'h' with cards in binary representation, i.e.,
    for h = ('Ac', 'Kd', 'Kh', 'Tc', '4d'), get binary representation
    using get_bin_rep function defined below. Here, binary representation is
    bh = (268471337, 134236965, 134228773, 16812055, 279045)
    
    >> get_hand_rank(bh)
    3570
    
    3570 is the unique rank for pair of Kings
    
    """
    q = reduce(lambda x,y: x|y, hand) >> 16
    if (reduce(lambda x,y: x&y, hand) & 0xF000) != 0:
        return flushes[q]
    elif q in uniques:
        return uniques[q]
    else:
        primes = [(c & 0xFF) for c in hand]
        r = reduce(lambda x,y: x*y, primes)
        return balance[r]

def get_bin_rep(card):
    r, s = card[0], card[1]
    return (ranks[r][0] << 16) | (suits[s][0] << 12) | (ranks[r][1] << 8) | ranks[r][2]

# Create deck
suits = {'c':[0b1000,'\u2663'],
         'd':[0b0100,'\u2662'],
         'h':[0b0010,'\u2661'],
         's':[0b0001,'\u2660']}

ranks = {'A': [0b0001000000000000, 12, 41],
         'K': [0b0000100000000000, 11, 37],
         'Q': [0b0000010000000000, 10, 31],
         'J': [0b0000001000000000,  9, 29],
         'T': [0b0000000100000000,  8, 23],
         '9': [0b0000000010000000,  7, 19],
         '8': [0b0000000001000000,  6, 17],
         '7': [0b0000000000100000,  5, 13],
         '6': [0b0000000000010000,  4, 11],
         '5': [0b0000000000001000,  3,  7],
         '4': [0b0000000000000100,  2,  5],
         '3': [0b0000000000000010,  1,  3],
         '2': [0b0000000000000001,  0,  2]}

deck = []
for i in ranks:
    for j in suits:
        deck.append(i+j)

bin_deck = [get_bin_rep(x) for x in deck]
bin_deck_dict = dict(zip(bin_deck, deck))

def get_pretty_suits(bin_card):
    card = bin_deck_dict[bin_card]
    return card[0]+suits[card[1]][1]

In [4]:
"""
Get hand ranks; apply primes ("p") and "qs" ("q") from ranks dict
per Cactus Kevin hash scheme
"""
df = pd.read_csv('poker hand equivalence classes.csv')
df['p'] = 1
df['q'] = 0
for i in range(5):
    df['p'] *= df['c'+str(i+1)].apply(lambda x: ranks[x][2])
    df['q'] |= df['c'+str(i+1)].apply(lambda x: ranks[x][0])

# Create ordered lookup tables for flushes, uniques, and balance of hands    
flush_mask = df.Hand.isin(['RF', 'SF', 'F'])
uniques_mask = df.Hand.isin(['S', 'HC'])

df_flushes = df.loc[flush_mask]
df_uniques = df.loc[uniques_mask]
df_balance = df.loc[~flush_mask & ~uniques_mask]

flush_dict = dict(zip(df_flushes.q, df_flushes.Rank))
flushes = dict(OrderedDict(sorted(flush_dict.items())))

uniques_dict = dict(zip(df_uniques.q, df_uniques.Rank))
uniques = dict(OrderedDict(sorted(uniques_dict.items())))

balance_dict = dict(zip(df_balance.p, df_balance.Rank))
balance = dict(OrderedDict(sorted(balance_dict.items())))

#### Choose number of players, deal hole cards, and compute pre-flop equity

In [40]:
# Deal hole cards
players = 6
hole_cards = random.sample(bin_deck, k=players*2)
avail_for_flop = [x for x in bin_deck if x not in hole_cards]

holes1 = hole_cards[:players]
holes2 = hole_cards[players:]
holes_by_player = [tuple(ele) for ele in list(zip(holes1, holes2))]

# Pretty suits
hole_cards_pretty = list(map(get_pretty_suits, hole_cards))
holes1p = hole_cards_pretty[:players]
holes2p = hole_cards_pretty[players:]
holes_by_player_pretty = [list(ele) for ele in list(zip(holes1p, holes2p))]

# Compute and print pre-flop equity using Monte Carlo sim
n_sims = 10**4
rank_array = np.zeros([n_sims, players])

for k in range(n_sims):
    community_cards = tuple(random.sample(avail_for_flop, k=5))

    for i in range(players):
        hole = holes_by_player[i]        
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+community_cards, 5):
            rank_list_iter.append(get_hand_rank(hand))

        rank_array[k, i] = min(rank_list_iter)
        
wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
preflop_equity = np.sum(wins_array/wins_total, axis=0)/k

print("Pre-flop equity:")
for j in range(players):
    print("{:5.0%}".format(preflop_equity[j]), "",
          holes_by_player_pretty[j])

Pre-flop equity:
  26%  ['T♣', 'Q♡']
  25%  ['4♠', 'A♢']
  11%  ['4♣', '8♡']
  11%  ['K♡', '2♡']
  19%  ['9♣', 'K♣']
   9%  ['5♣', '2♣']


#### Post-flop

In [41]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_flop = choose(52-2*players-3, 2)
rank_array = np.zeros([combos_post_flop, players])

flop = tuple(random.sample(avail_for_flop, k=3))
avail_for_turn = [x for x in avail_for_flop if x not in flop]

print("Flop:", list(map(get_pretty_suits, flop)), "\n")

for k, turn_and_river in enumerate(combinations(avail_for_turn, 2)):
    for i in range(players):
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+flop+turn_and_river, 5):
            rank_list_iter.append(get_hand_rank(hand))
        
        rank_array[k, i] = min(rank_list_iter)

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postflop_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_flop
        
print("Post-flop equity and improvement vs. pre-flop:")
for j in range(players):
    print("{:5.0%}".format(postflop_equity[j]), 
          "{:+5.0%}".format(postflop_equity[j]-preflop_equity[j]), "",
          holes_by_player_pretty[j])

Flop: ['9♡', '8♣', 'J♠'] 

Post-flop equity and improvement vs. pre-flop:
  88%  +62%  ['T♣', 'Q♡']
   0%  -25%  ['4♠', 'A♢']
   2%   -9%  ['4♣', '8♡']
   5%   -6%  ['K♡', '2♡']
   6%  -14%  ['9♣', 'K♣']
   0%   -9%  ['5♣', '2♣']


#### Post-turn, pre-river

In [42]:
# Will be exact equity as # of combos small enough to not have to use Monte Carlo
combos_post_turn = 52-2*players-3-1
rank_array = np.zeros([combos_post_turn, players])

turn = tuple(random.sample(avail_for_turn, k=1))
avail_for_river = [x for x in avail_for_turn if x not in turn]

print("Flop:", list(map(get_pretty_suits, flop)))
print("Turn:", list(map(get_pretty_suits, turn)), "\n")

for k, river in enumerate(combinations(avail_for_river, 1)):
    for i in range(players):
        rank_list_iter = []
        
        for hand in combinations(holes_by_player[i]+flop+turn+river, 5):
            rank_list_iter.append(get_hand_rank(hand))
        
        rank_array[k, i] = min(rank_list_iter)

wins_array = rd(rank_array, axis=1, method='min')==1
wins_total = (rd(rank_array, axis=1, method='min')==1).sum(axis=1).reshape(-1,1)
postturn_equity = np.sum(wins_array/wins_total, axis=0)/combos_post_turn

print("Post-turn equity and improvement vs. flop:")
for j in range(players):
    print("{:5.0%}".format(postturn_equity[j]), 
          "{:+5.0%}".format(postturn_equity[j]-postflop_equity[j]), "",
          holes_by_player_pretty[j])

Flop: ['9♡', '8♣', 'J♠']
Turn: ['T♠'] 

Post-turn equity and improvement vs. flop:
  92%   +4%  ['T♣', 'Q♡']
   0%   +0%  ['4♠', 'A♢']
   0%   -2%  ['4♣', '8♡']
   4%   -1%  ['K♡', '2♡']
   4%   -2%  ['9♣', 'K♣']
   0%   +0%  ['5♣', '2♣']


#### Post-river final hand rank

In [43]:
rank_array = np.zeros([1, players])

river = tuple(random.sample(avail_for_river, k=1))

print("Flop:", list(map(get_pretty_suits, flop)))
print("Turn:", list(map(get_pretty_suits, turn)))
print("River:", list(map(get_pretty_suits, river)), "\n")

for i in range(players):
    rank_list_iter = []
    
    for hand in combinations(holes_by_player[i]+flop+turn+river, 5):
        rank_list_iter.append(get_hand_rank(hand))
    
    rank_array[0, i] = min(rank_list_iter)
        
print("Final hand rankings and improvement vs. turn:")
for j in range(players):
    print("{:5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)), 
          "{:+5.0%}".format(np.sum(rank_array.argmin(axis=1)==j)-postturn_equity[j]), "",
          holes_by_player_pretty[j])

print("\n", "Winning hand:",
      df.loc[df.Rank==np.min(rank_array), 'HandName'].to_string(index=False))

Flop: ['9♡', '8♣', 'J♠']
Turn: ['T♠']
River: ['J♢'] 

Final hand rankings and improvement vs. turn:
 100%   +8%  ['T♣', 'Q♡']
   0%   +0%  ['4♠', 'A♢']
   0%   +0%  ['4♣', '8♡']
   0%   -4%  ['K♡', '2♡']
   0%   -4%  ['9♣', 'K♣']
   0%   +0%  ['5♣', '2♣']

 Winning hand:  Queen-High Straight
